<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/dev/Archivo_de_cierre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías necesarias

In [13]:
'''pip install xlsxwriter'''

'pip install xlsxwriter'

Nota: openpyxl ya está por defecto en colab

In [14]:
import pandas as pd
import xlsxwriter
import numpy as np
from openpyxl import Workbook
from openpyxl import load_workbook

# Limpieza y transformación de SICAR Ventas

In [15]:
ventas = pd.read_excel('Ventas.xlsx', sheet_name=0, header=5)
ventas.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Documento,Unnamed: 1,Unnamed: 2,Fecha,Unnamed: 4,Folio,Unnamed: 6,Cliente,Unnamed: 8,Unnamed: 9,...,Total,Unnamed: 31,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Ticket,NaN,NaN,2025-07-03 08:03:19,NaN,63015.0,NaN,Público en General,NaN,NaN,...,$ 49.00,NaN,$ 49.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
1,PZA,1.0,NaN,NaT,[7501206674994] 21531 Brocha uso general de 6'...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ticket,NaN,NaN,2025-07-03 08:15:21,NaN,63016.0,NaN,Público en General,NaN,NaN,...,$ 119.00,NaN,$ 119.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
3,PZA,1.0,NaN,NaT,[7506240645290] 13882 Felpa para rodillo 9' x ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PZA,1.0,NaN,NaT,[7501206619223] 20323 Espátula flexible 5' con...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Explicación 1

- "Unnamed: \d+": Es un patrón de expresión regular que coincida con la cadena "Unnamed: " seguido de uno o más digitos (\d+). Este patrón suele ser usado en excels que tienen columnas sin nombre.

- regex=True: Especifica que el primer argumento es una expresión regular.

Anexos:
- https://stackoverflow.com/questions/2841550/what-does-d-mean-in-a-regular-expression

In [16]:
ventas.columns = ventas.columns.str.replace("Unnamed: \d+", "", regex=True)

columnas = ventas.columns.to_list()
columnas

['Documento',
 '',
 '',
 'Fecha',
 '',
 'Folio',
 '',
 'Cliente',
 '',
 '',
 '',
 '',
 'Caja',
 '',
 '',
 '',
 '',
 'Usuario',
 '',
 '',
 '',
 '',
 '',
 'Folio F.',
 '',
 '',
 'Est',
 '',
 '',
 '',
 'Total   ',
 '',
 'Efectivo',
 'Cheque',
 'Crédito',
 'Transferencia',
 'Vales',
 'Tarjeta',
 'Anticipo',
 'SICAR Pagos']

## Explicación 2

Filtra las cadenas vacías de la lista de columnas. Revisar el archivo origen de SICAR

In [17]:
columnas_con_nombre = [col for col in columnas if col != '']

In [18]:
ventas = ventas[columnas_con_nombre]
ventas.head()

,Documento,Fecha,Folio,Cliente,Caja,Usuario,Folio F.,Est,Total,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Ticket,2025-07-03 08:03:19,63015.0,Público en General,Caja 1,admin,NaN,V,$ 49.00,$ 49.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
1,PZA,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ticket,2025-07-03 08:15:21,63016.0,Público en General,Caja 1,admin,NaN,V,$ 119.00,$ 119.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
3,PZA,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PZA,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


No hacemos un drop.na por la columna de "Folio F.", en donde la mayoría de los datos son nulos

In [19]:
ventas = ventas.loc[(ventas["Documento"] == "Ticket") | (ventas["Documento"] == "Factura CFDI")]
ventas.head()

,Documento,Fecha,Folio,Cliente,Caja,Usuario,Folio F.,Est,Total,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Ticket,2025-07-03 08:03:19,63015.0,Público en General,Caja 1,admin,NaN,V,$ 49.00,$ 49.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
2,Ticket,2025-07-03 08:15:21,63016.0,Público en General,Caja 1,admin,NaN,V,$ 119.00,$ 119.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
7,Ticket,2025-07-03 08:19:15,63017.0,Público en General,Caja 1,admin,NaN,V,$ 12.00,$ 12.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
9,Factura CFDI,2025-07-03 08:22:30,22736.0,VISION COMERCIAL DEPORTIVA,Caja 1,admin,22736.0,V,$ 279.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 279.00,$ 0.00,$ 0.00
12,Ticket,2025-07-03 08:26:35,63018.0,Público en General,Caja 1,admin,NaN,V,$ 326.00,$ 326.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00


## Explicación 3
Eliminamos todos los signos y comas de las columnas que contienen información numérica. Si no, no podemos hacer los cálculos necesarios.

Nota: La columna "Total" del archivo de ventas de SICAR tiene esos espacios en blanco.

In [20]:
ventas['Total   '] = ventas["Total   "].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Efectivo'] = ventas["Efectivo"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Cheque'] = ventas["Cheque"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Crédito'] = ventas["Crédito"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Transferencia'] = ventas["Transferencia"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Vales'] = ventas["Vales"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Tarjeta'] = ventas["Tarjeta"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Anticipo'] = ventas["Anticipo"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['SICAR Pagos'] = ventas["SICAR Pagos"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)


In [21]:
ventas.head()

,Documento,Fecha,Folio,Cliente,Caja,Usuario,Folio F.,Est,Total,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Ticket,2025-07-03 08:03:19,63015.0,Público en General,Caja 1,admin,NaN,V,49.0,49.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Ticket,2025-07-03 08:15:21,63016.0,Público en General,Caja 1,admin,NaN,V,119.0,119.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Ticket,2025-07-03 08:19:15,63017.0,Público en General,Caja 1,admin,NaN,V,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Factura CFDI,2025-07-03 08:22:30,22736.0,VISION COMERCIAL DEPORTIVA,Caja 1,admin,22736.0,V,279.0,0.0,0.0,0.0,0.0,0.0,279.0,0.0,0.0
12,Ticket,2025-07-03 08:26:35,63018.0,Público en General,Caja 1,admin,NaN,V,326.0,326.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Sacamos los totales de todas las columnas de dinero

In [22]:
total = ventas['Total   '].sum().round(2)
total

np.float64(64234.8)

In [23]:
efectivo = ventas['Efectivo'].sum().round(2)
efectivo

np.float64(23590.2)

In [24]:
cheque = ventas['Cheque'].sum().round(2)
cheque

np.float64(74.0)

In [25]:
credito = ventas['Crédito'].sum().round(2)
credito

np.float64(0.0)

In [26]:
transferencia = ventas['Transferencia'].sum().round(2)
transferencia

np.float64(8083.0)

In [27]:
vales = ventas['Vales'].sum().round(2)
vales

np.float64(0.0)

In [28]:
tarjeta = ventas['Tarjeta'].sum().round(2)
tarjeta

np.float64(32487.6)

In [29]:
anticipo = ventas['Anticipo'].sum().round(2)
anticipo

np.float64(0.0)

In [30]:
s_pagos = ventas['SICAR Pagos'].sum().round(2)
s_pagos

np.float64(0.0)

# Limpieza y transformación de SICAR Descuentos

In [31]:
descuentos = pd.read_excel("Descuentos.xlsx", sheet_name=0, header=9)
descuentos.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Documento,Unnamed: 1,Fecha,Unnamed: 3,Unnamed: 4,Folio,Unnamed: 6,Cliente,Unnamed: 8,Unnamed: 9,...,Unnamed: 23,Unnamed: 24,Descuento $\n,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Total
0,Ticket,NaN,2025-07-03 08:03:19,NaN,NaN,63015.0,NaN,Público en General,NaN,NaN,...,NaN,NaN,$ 0.00,NaT,NaN,NaN,NaN,NaN,NaN,$ 49.00
1,NaN,NaN,NaT,NaN,NaN,NaN,PZA,NaN,1.0,[7501206674994] 21531 Brocha uso general de 6'...,...,$ 49.00,NaN,NaN,NaT,NaN,NaN,$ 49.00,NaN,NaN,NaN
2,Ticket,NaN,2025-07-03 08:15:21,NaN,NaN,63016.0,NaN,Público en General,NaN,NaN,...,NaN,NaN,$ 0.00,NaT,NaN,NaN,NaN,NaN,NaN,$ 119.00
3,NaN,NaN,NaT,NaN,NaN,NaN,PZA,NaN,1.0,[7506240645290] 13882 Felpa para rodillo 9' x ...,...,$ 22.00,NaN,NaN,NaT,NaN,NaN,$ 22.00,NaN,NaN,NaN
4,NaN,NaN,NaT,NaN,NaN,NaN,PZA,NaN,1.0,[7501206619223] 20323 Espátula flexible 5' con...,...,$ 25.00,NaN,NaN,NaT,NaN,NaN,$ 25.00,NaN,NaN,NaN


Nota: Ver "Explicación 1"

In [32]:
descuentos.columns = descuentos.columns.str.replace("Unnamed: \d+", "", regex=True)

columnas = descuentos.columns.to_list()
columnas

['Documento',
 '',
 'Fecha',
 '',
 '',
 'Folio',
 '',
 'Cliente',
 '',
 '',
 '',
 '',
 '',
 'Caja',
 '',
 '',
 'Usuario',
 '',
 '',
 'Est',
 '',
 'Desc %',
 '',
 '',
 '',
 'Descuento $\n',
 '',
 '',
 '',
 '',
 '',
 '',
 'Total']

Nota: Ver explicación 2

In [33]:
columnas_con_nombre = [col for col in columnas if col != '']

columnas_con_nombre

['Documento',
 'Fecha',
 'Folio',
 'Cliente',
 'Caja',
 'Usuario',
 'Est',
 'Desc %',
 'Descuento $\n',
 'Total']

In [34]:
descuentos = descuentos[columnas_con_nombre]
descuentos.head()

,Documento,Fecha,Folio,Cliente,Caja,Usuario,Est,Desc %,Descuento $\n,Total
0,Ticket,2025-07-03 08:03:19,63015.0,Público en General,Caja 1,admin,V,0.00%,$ 0.00,$ 49.00
1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ticket,2025-07-03 08:15:21,63016.0,Público en General,Caja 1,admin,V,0.00%,$ 0.00,$ 119.00
3,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
descuentos = descuentos.loc[(descuentos["Documento"] == "Ticket") | (descuentos["Documento"] == "Factura CFDI")]
descuentos.head()

,Documento,Fecha,Folio,Cliente,Caja,Usuario,Est,Desc %,Descuento $\n,Total
0,Ticket,2025-07-03 08:03:19,63015.0,Público en General,Caja 1,admin,V,0.00%,$ 0.00,$ 49.00
2,Ticket,2025-07-03 08:15:21,63016.0,Público en General,Caja 1,admin,V,0.00%,$ 0.00,$ 119.00
7,Ticket,2025-07-03 08:19:15,63017.0,Público en General,Caja 1,admin,V,0.00%,$ 0.00,$ 12.00
9,Factura CFDI,2025-07-03 08:22:30,22736.0,VISION COMERCIAL DEPORTIVA,Caja 1,admin,V,0.00%,$ 0.00,$ 279.00
12,Ticket,2025-07-03 08:26:35,63018.0,Público en General,Caja 1,admin,V,0.00%,$ 0.00,$ 326.00


Nota: Ver explicación 3

In [36]:
descuentos['Total'] = descuentos["Total"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
descuentos['Descuento $\n'] = descuentos["Descuento $\n"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)

In [37]:
descuentos.head()

,Documento,Fecha,Folio,Cliente,Caja,Usuario,Est,Desc %,Descuento $\n,Total
0,Ticket,2025-07-03 08:03:19,63015.0,Público en General,Caja 1,admin,V,0.00%,0.0,49.0
2,Ticket,2025-07-03 08:15:21,63016.0,Público en General,Caja 1,admin,V,0.00%,0.0,119.0
7,Ticket,2025-07-03 08:19:15,63017.0,Público en General,Caja 1,admin,V,0.00%,0.0,12.0
9,Factura CFDI,2025-07-03 08:22:30,22736.0,VISION COMERCIAL DEPORTIVA,Caja 1,admin,V,0.00%,0.0,279.0
12,Ticket,2025-07-03 08:26:35,63018.0,Público en General,Caja 1,admin,V,0.00%,0.0,326.0


In [38]:
descuento = descuentos['Descuento $\n'].sum().round(2)
descuento

np.float64(107.4)

# Extracción ARQUEO

Consultar el archivo de "Cierre.xlsx"

In [39]:
wb = load_workbook('Arqueo.xlsx', data_only=True)
ws = wb.get_sheet_by_name('03JULIO')

/tmp/ipython-input-39-3248029377.py:2: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  ws = wb.get_sheet_by_name('03JULIO')


In [40]:
inicio_caja = ws['L10'].value
inicio_caja

800

In [41]:
t_monedas_20 = 0
t_monedas_50 = 0
t_monedas1 = 0
t_monedas2 = 0
t_monedas5 = 0
t_monedas10 = 0

In [42]:
try:
  t_monedas_20 += ws['E28'].value
except:
  t_monedas_20 += 0
try:
  t_monedas_20 += ws['E42'].value
except:
  t_monedas_20 += 0
try:
  t_monedas_20 += ws['E56'].value
except:
  t_monedas_20 += 0

print(t_monedas_20)

0


In [43]:
try:
  t_monedas_50 += ws['E29'].value
except:
  t_monedas_50 += 0
try:
  t_monedas_50 += ws['E43'].value
except:
  t_monedas_50 += 0
try:
  t_monedas_50 += ws['E57'].value
except:
  t_monedas_50 += 0

print(t_monedas_50)

0


In [44]:
try:
  t_monedas1 += ws['E30'].value
except:
  t_monedas1 += 0
try:
  t_monedas1 += ws['E44'].value
except:
  t_monedas1 += 0
try:
  t_monedas1 += ws['E58'].value
except:
  t_monedas1 += 0

print(t_monedas1)

50


In [45]:
try:
  t_monedas2 += ws['E31'].value
except:
  t_monedas2 += 0
try:
  t_monedas2 += ws['E45'].value
except:
  t_monedas2 += 0
try:
  t_monedas2 += ws['E59'].value
except:
  t_monedas2 += 0

print(t_monedas2)

try:
  t_monedas5 += ws['E32'].value
except:
  t_monedas5 += 0
try:
  t_monedas5 += ws['E46'].value
except:
  t_monedas5 += 0
try:
  t_monedas5 += ws['E60'].value
except:
  t_monedas5 += 0

print(t_monedas5)

try:
  t_monedas10 += ws['E33'].value
except:
  t_monedas10 += 0
try:
  t_monedas10 += ws['E47'].value
except:
  t_monedas10 += 0
try:
  t_monedas10 += ws['E61'].value
except:
  t_monedas10 += 0

t_monedas10

20
20


10

In [46]:
t_billetes20 = 0
t_billetes50 = 0
t_billetes100 = 0
t_billetes200 = 0
t_billetes500 = 0
t_billetes1000 = 0

In [47]:
t_billetes20 = 0
try:
  t_billetes20 += ws['I28'].value
except:
  t_billetes20 += 0
try:
  t_billetes20 += ws['I42'].value
except:
  t_billetes20 += 0
try:
  t_billetes20 += ws['I56'].value
except:
  t_billetes20 += 0

t_billetes20

5

In [48]:
# prompt: crea una lógica similar para t_billetes50, t_billetes100, t_billetes200, t_billetes500 y t_billetes1000 siempre usando la siguiente columna

t_billetes50 = 0
try:
  t_billetes50 += ws['I29'].value
except:
  t_billetes50 += 0
try:
  t_billetes50 += ws['I43'].value
except:
  t_billetes50 += 0
try:
  t_billetes50 += ws['I57'].value
except:
  t_billetes50 += 0


t_billetes100 = 0
try:
  t_billetes100 += ws['I30'].value
except:
  t_billetes100 += 0
try:
  t_billetes100 += ws['I44'].value
except:
  t_billetes100 += 0
try:
  t_billetes100 += ws['I58'].value
except:
  t_billetes100 += 0


t_billetes200 = 0
try:
  t_billetes200 += ws['I31'].value
except:
  t_billetes200 += 0
try:
  t_billetes200 += ws['I45'].value
except:
  t_billetes200 += 0
try:
  t_billetes200 += ws['I59'].value
except:
  t_billetes200 += 0


t_billetes500 = 0
try:
  t_billetes500 += ws['I32'].value
except:
  t_billetes500 += 0
try:
  t_billetes500 += ws['I46'].value
except:
  t_billetes500 += 0
try:
  t_billetes500 += ws['I60'].value
except:
  t_billetes500 += 0


t_billetes1000 = 0
try:
  t_billetes1000 += ws['I33'].value
except:
  t_billetes1000 += 0
try:
  t_billetes1000 += ws['I47'].value
except:
  t_billetes1000 += 0
try:
  t_billetes1000 += ws['I61'].value
except:
  t_billetes1000 += 0


In [49]:
print("t_billetes20:", t_billetes20)
print("t_billetes50:", t_billetes50)
print("t_billetes100:", t_billetes100)
print("t_billetes200:", t_billetes200)
print("t_billetes500:", t_billetes500)
print("t_billetes1000:", t_billetes1000)

t_billetes20: 5
t_billetes50: 4
t_billetes100: 4
t_billetes200: 4
t_billetes500: 23
t_billetes1000: 1


In [50]:
t_m_20 = t_monedas_20 * 0.20
t_m_50 = t_monedas_50 * 0.50
t_m1 = t_monedas1
t_m2 = t_monedas2 * 2
t_m5 = t_monedas5 * 5
t_m10 = t_monedas10 * 10

In [51]:
t_b20 = t_billetes20 * 20
t_b50 = t_billetes50 * 50
t_b100 = t_billetes100 * 100
t_b200 = t_billetes200 * 200
t_b500 = t_billetes500 * 500
t_b1000 = t_billetes1000 * 1000

In [52]:
total_monedas = t_m_20 + t_m_50 + t_m1 + t_m2 + t_m5 + t_m10
total_billetes = t_b20 + t_b50 + t_b100 + t_b200 + t_b500 + t_b1000
total_caja = total_monedas + total_billetes

In [53]:
try:
  getnet = ws["K120"].value
except:
  getnet = 0
try:
  evo = ws["H120"].value
except:
  evo = 0
try:
  a_express = ws["E120"].value
except:
  a_express = 0
try:
  m_pago = ws["N120"].value
except:
  m_pago = 0

In [54]:
total_terminales = (getnet if getnet is not None else 0) + (evo if evo is not None else 0) + (a_express if a_express is not None else 0) + (m_pago if m_pago is not None else 0)
total_terminales

32476.5

In [55]:
caja_inicial = ws["L10"].value
salidas_en_efectivo = 0
caja_abono = 0
venta_abono = 0
salidas_abono = ws['L89'].value

In [56]:
totales_cargo = efectivo + caja_inicial + salidas_en_efectivo
totales_abono = caja_abono + venta_abono + salidas_abono

In [57]:
caja_de_cambio = ws["L140"].value

In [58]:
saldo = totales_cargo - totales_abono
saldo_efectivo = total_caja - saldo